source: https://medium.com/@lucafiaschi/turning-customers-feedback-into-action-an-llm-blueprint-for-app-review-analysis-7f5d39d08f6e

In [11]:
import json
import pandas as pd
import numpy as np

from joblib import Parallel, delayed
import datetime
import time

import openai
import os

from yaml_helper import YamlParser

In [4]:
df = pd.read_csv("data/Airline_review.csv")

In [8]:
df.sample(1000).to_csv("data/Airline_review_sample.csv", index=False)

In [12]:
creds = YamlParser("creds/openai.yml").read()
OPENAI_API_KEY = creds["OPENAI_API_KEY"]


In [13]:
def retry_on_error(max_retries=5, sleep_time=5):
    """
    Handy decorator to fight the crashes in the API
    """
    def decorator_retry(func):
        def wrapper(*args, **kwargs):
            retries = 0
            while retries < max_retries:
                try:
                    result = func(*args, **kwargs)
                    return result
                except Exception as e:
                    print(f"An error occurred: {e}")
                    retries += 1
                    time.sleep(sleep_time*np.random.rand())
            return None
        return wrapper
    return decorator_retry

In [ ]:
@retry_on_error()
def classify_topic(text):
    openai.api_key = OPENAI_API_KEY
    prompt = MARKDOWN_PROMPT%(text)

    client = openai.OpenAI(api_key=OPENAI_API_KEY)
    response = client.chat.completions.create(
            model="gpt-4.1",
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=2048,
            n=1,
            temperature=0.1,
        )

    result = json.loads(response.choices[0].message.content.strip())
    result['content'] = text
    return result

In [ ]:
res=Parallel(n_jobs=25,verbose=2)(delayed(classify_topic)(text) for text in df.content)

df_result = pd.DataFrame(res)